# Tải các thư viện cần thiết

In [18]:
! pip install torch

In [19]:
!pip install nltk pyvi

In [20]:
!pip install -q spacy emoji py_vncorenlp

!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 13.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


# Chuyển file data crawl được dưới dạng txt sang dạng json

In [5]:
# Upload file data.txt chứa mẫu dữ liệu crawl được
from google.colab import files
uploaded = files.upload()


Saving claim.txt to claim.txt


In [6]:
# Chuyển file txt sang định dạng file json
import json

txt_filename = 'claim.txt'
json_filename = 'claim.json'

claims = []

# Đọc file txt và chuyển từng dòng thành dict
with open(txt_filename, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            claims.append({"claim": line})

# Ghi ra file JSON
with open(json_filename, 'w', encoding='utf-8') as f:
    json.dump(claims, f, ensure_ascii=False, indent=2)

print(f"Đã chuyển {len(claims)} dòng thành JSON và lưu vào '{json_filename}'")

Đã chuyển 2008 dòng thành JSON và lưu vào 'claim.json'


# Tiền xử lý dữ liệu

## Loại bỏ emoji

In [7]:
import json
import re
import string

def remove_emoji(string):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"
        u"\U0001F300-\U0001F5FF"
        u"\U0001F680-\U0001F6FF"
        u"\U0001F1E0-\U0001F1FF"
        u"\U00002500-\U00002BEF"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"
        u"\u3030"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

## Loại bỏ emoticon ký tự

In [8]:
def remove_emoticons(text):
    emoticon_pattern = re.compile(r'(:\)+|:\(+|=+\)+|:v|:V|:\||:D|:\>|\:<|:3|:P|:p|:o|:O|:s|:S|:x|:X|:’\(|:")')
    return emoticon_pattern.sub('', text)

## Loại bỏ Tag tên

In [9]:
def remove_tag_name(text):
    return re.sub(r'@(?:[A-ZÀ-Ỵ][a-zà-ỹ]*\s?)+,?', '', text)

## Loại bỏ Liên kết

In [10]:
def remove_links(text):
    return re.sub(r"http\S+|www\S+", '', text)

## Loại bỏ các ký tự lặp liên tiếp từ 3 lần trở lên thành 2 lần

In [11]:
def delete_redundant_characters(text):
    return re.sub(r'(.)\1{2,}', r'\1\1', text)

## Chuẩn hóa khoảng trắng

In [12]:
def normalize_whitespace(text):
    return re.sub(r'\s+', ' ', text).strip()

## Xử lý Teencode, Từ tiếng Anh

In [23]:
teencode_dict = {
    "t" : "tôi",
    "tao" : "tôi",
    "tui" : "tôi",
    "m" : "mày",
    "K" : "không",
    "k" : "không",
    "ko" : "không",
    "nhma" : "nhưng mà",
    "b" : "bạn",
    "h" : "giờ",
    "z" : "vậy",
    "zị" : "vậy",
    "v" : "vậy",
    "ng" : "người",
    "ngta" : "người ta",
    "mn" : "mọi người",
    "ms" : "mới",
    "dc" : "được",
    "đc" : "được",
    "r" : "rồi",
    "bl" : "bình luận",
    "vk" : "vợ",
    "ck" : "chồng",
    "thg" : "thằng",
    "CA" : "công an",
    "e" : "em",
    "vs" : "với",
    "ae" : "anh em",
    "ntn" : "như thế này",
    "bt" : "biết",
    "j" : "gì",
    "cgi" : "cái gì",
    "fb" : "facebook",
    "cmt" : "bình luận",
    "tks" : "cảm ơn",
    "dt" : "dân tộc",
    "vn" : "Việt Nam",
    "VN" : "Việt Nam",
    "sg" : "Thành phố Hồ Chí Minh",
    "xh" : "xã hội",
    "m" : "mày",
    "bn" : "bạn",
    "a" : "anh",
    "e": "em",
    "mk": "mình",
    "mik": "mình",
    "ngkh": "người khác",
    "mxh" : "mạng xã hội",
    "gđ" : "gia đình",
    "đh" : "Đại học",
    "Đh" : "Đại học",
    "cntt" : "Công nghệ thông tin",
    "Cntt" : "Công nghệ thông tin",
    "auto" : "hiển nhiên",
    "bro" : "anh em",
    "thui" : "thôi",
    "tr": "trời",
    "kcn": "kem chống nắng",
    "sk" : "sức khỏe",
    "TQ" : "Trung Quốc",
    "mấy fen" : "các bạn",
    "uk" : "ừ",
    "đr" : "đúng rồi",
    "ykr" : "ý kiến riêng",
    "đv" : "đối với",
    "ròi" : "rồi",
    "rùi" : "rồi",
    "mọi ng" : "mọi người",
    "hic" : "",
    "huhu" : "",
    "iu": "yêu",
    "tóp tóp": "tiktok",
    "zậy": "vậy",
    "zồi": "rồi",
    "hok" : "không",
    "ơi zời" : "ôi trời",
    "zị" : "vậy",
    "chj" : "chị",
    "zai" : "trai",
    "SVĐ" : "sân vận động",
    "HN" : "Hà Nội",
    "thk" : "thằng",
    "nc" : "nước",
    "vs" : "với",
    "TópTóp" : "TikTok",
    "nhìu" : "nhiều",
    "ng" : "người",
    "p" : "phút",
    "bsĩ" : "bác sĩ",
    "hk" : "không",
    "zui zẻ" : "vui vẻ",
    "SV" : "sinh viên",
    "tỏi" : "tỷ",
    "TPHCM" : "Thành phố Hồ Chí Minh",
    "GV" : "giáo viên",
    "củ" : "triệu",
    "Clb" : "câu lạc bộ",
    "KHXH&NV" : "Khoa học xã hội và Nhân văn",
    "dzị" : "vậy",
    "GTVT" : "Giao thông vận tải",
    "lag": "giật",
    "Mẹc": "Mercedes",
    "ĐHQGHN": "Đại học Quốc gia Hà Nội",
    "10k": "10 nghìn",
    "ĐHQG TPHCM": "Đại học Quốc gia Thành phố Hồ Chí Minh",
    "GDQP": "Giáo dục Quốc phòng",
    "info": "thông tin",
    "ĐH BKHN": "Đại học Bách khoa Hà Nội",
    "KTX": "ký túc xá",
    "TDTT": "Thể dục thể thao",
    "acc clone": "tài khoản phụ",
    "spam": "tin rác",
    "update": "cập nhật",
    "content": "nội dung",
    "subs": "người theo dõi",
    "template": "mẫu",
    "text": "văn bản",
    "newsfeed": "bảng tin",
    "apply": "nộp đơn",
    "N2": "trình độ tiếng Nhật N2",
    "Fresher": "người mới",
    "skill": "kỹ năng",
    "HR": "nhân sự",
    "TNCN": "thu nhập cá nhân",
    "Outsource": "thuê ngoài",
    "networking": "xây dựng mối quan hệ",
    "FMCG": "hàng tiêu dùng nhanh",
    "OT": "tăng ca",
    "Broker": "môi giới",
    "ads": "quảng cáo",
    "kk" : "",
    "BĐS" : "bất động sản"
}

In [24]:
def replace_teencode(text, teencode_dict):
    if not isinstance(text, str):
        return text

    # Sắp xếp theo độ dài từ khóa để xử lý cụm trước
    sorted_dict = dict(sorted(teencode_dict.items(), key=lambda x: -len(x[0])))

    for word, replacement in sorted_dict.items():
        pattern = r'\b' + re.escape(word) + r'\b'
        text = re.sub(pattern, replacement, text, flags=re.IGNORECASE)
    return text

## Áp dụng cho file json dữ liệu

In [25]:
def preprocess_claim(text, teencode_dict):
    text = remove_emoji(text)
    text = remove_emoticons(text)
    text = remove_tag_name(text)
    text = remove_links(text)
    text = delete_redundant_characters(text)
    text = replace_teencode(text, teencode_dict)
    text = normalize_whitespace(text)
    return text

In [26]:
def process_json_claims(input_path, output_path, teencode_dict):
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    for item in data:
        if "claim" in item:
            item["claim"] = preprocess_claim(item["claim"], teencode_dict)

    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

    print(f"Đã xử lý xong và lưu vào '{output_path}'")

In [27]:
process_json_claims("claim.json", "claim_cleaned.json", teencode_dict)

Đã xử lý xong và lưu vào 'claim_cleaned.json'
